### Imports

In [1893]:
import pandas as pd

### Read Data

In [1894]:
downtimeNew = pd.read_excel('Raw Data/Equipment downtime data (202310).xlsx')
downtimeNew.to_csv('Raw Data/Equipment downtime data (202310).csv', index=False)
sensorsTempRead = pd.read_csv('Sensor Data/irfq__boiler_read_temp-data-11_11_2023, 22_56_28.csv')
h2GasFlow = pd.read_csv('Sensor Data/irfq__h2_gas_read_flow-data-11_11_2023, 23_19_19.csv')
sensorVolts = pd.read_csv('Sensor Data/irfq__ext_on + irfq__ext_read_current + irfq__ext_read_volts + irfq__ext_set_volts-data-as-joinbyfield-11_11_2023, 23_16_24.csv')

In [1895]:
downtimeNew['FaultDate'] = pd.to_datetime(downtimeNew['FaultDate'], errors='coerce').dt.date
downtimeNew['FaultTime'] = downtimeNew['FaultTime'].apply(lambda x: x.strftime('%H:%M:%S') if len(str(x)) > 8 else x)
downtimeNew = downtimeNew.dropna(subset=['FaultDate', 'FaultTime'])
pattern_new = downtimeNew.iloc[:, :14]
pattern_new = pattern_new.drop(['ID', 'DutyOfficer', 'Manager email address'], axis=1)
pattern_new['FaultDateTime'] = pd.to_datetime(pattern_new['FaultDate'].astype(str) + ' ' + pattern_new['FaultTime'].astype(str))
pattern_new.drop(['FaultDate', 'FaultTime'], axis=1, inplace=True)
pattern_new.sort_values(by=['FaultDateTime'], inplace=True)
pattern_new = pattern_new.drop(['LogEntry', 'DutyOfficer comments', 'Managerscomments', 'FaultRepair', 'FaultDescription', 'Group', 'Downtime'], axis=1)
pattern_new['Equipment'] = pattern_new['Equipment'].str.lower()
pattern_new['Equipment'] = pattern_new['Equipment'].str.replace('[^\w\s]', '')

/var/folders/nl/8bkjfnbd2k53t1p3_gjy6k9h0000gn/T/ipykernel_42092/3142707672.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  pattern_new['Equipment'] = pattern_new['Equipment'].str.replace('[^\w\s]', '')


In [1896]:
pattern_new = pattern_new[pattern_new['User Run'] == '23/03']
pattern_new.drop(['User Run'], axis=1, inplace=True)

In [1897]:
failed = pattern_new[pattern_new['Equipment'] == 'ion source']

In [1898]:
sensorVolts.drop(["irfq::ext:on","irfq::ext:read_current","irfq::ext:set_volts"], axis=1, inplace=True)
sensorVolts.dropna(subset=['irfq::ext:read_volts'], inplace=True)

In [1899]:
sensorsTempRead['Time'] = pd.to_datetime(sensorsTempRead['Time'], format="%d/%m/%Y, %H:%M:%S")
h2GasFlow['Time'] = pd.to_datetime(h2GasFlow['Time'], format="%d/%m/%Y, %H:%M:%S")
sensorVolts['Time'] = pd.to_datetime(sensorVolts['Time'], format="%d/%m/%Y, %H:%M:%S")

In [1891]:
sensorsTempRead['Date'] = sensorsTempRead['Time'].dt.date
sensorsTempRead['Hour'] = sensorsTempRead['Time'].dt.hour
# sensorsTempRead['Minute'] = sensorsTempRead['Time'].dt.minute
sensorsTempRead.drop(['Time'], axis=1, inplace=True)

h2GasFlow['Date'] = h2GasFlow['Time'].dt.date
h2GasFlow['Hour'] = h2GasFlow['Time'].dt.hour
# h2GasFlow['Minute'] = h2GasFlow['Time'].dt.minute
h2GasFlow.drop(['Time'], axis=1, inplace=True)

sensorVolts['Date'] = sensorVolts['Time'].dt.date
sensorVolts['Hour'] = sensorVolts['Time'].dt.hour
# sensorVolts['Minute'] = sensorVolts['Time'].dt.minute
sensorVolts.drop(['Time'], axis=1, inplace=True)

Look at n readings. If fail in timestamp in readings range label = 1

In [1862]:
hr_avg = sensorsTempRead.groupby(['Date', 'Hour'])['irfq::boiler:read_temp'].mean()
hr_avg = hr_avg.reset_index()
hr_avg['Datetime'] = hr_avg['Date'].astype(str) + ' ' + hr_avg['Hour'].astype(str) + ':00:00'
hr_avg['Datetime'] = pd.to_datetime(hr_avg['Datetime'], format="%Y-%m-%d %H:%M:%S")

# -----------------------------

hr_avg_g = h2GasFlow.groupby(['Date', 'Hour'])['irfq::h2_gas:read_flow'].mean()
hr_avg_g = hr_avg_g.reset_index()
hr_avg_g['Datetime'] = hr_avg_g['Date'].astype(str) + ' ' + hr_avg_g['Hour'].astype(str) + ':00:00'
hr_avg_g['Datetime'] = pd.to_datetime(hr_avg_g['Datetime'], format="%Y-%m-%d %H:%M:%S")

# -----------------------------

avg_volts = sensorVolts.groupby(['Date', 'Hour'])['irfq::ext:read_volts'].mean()
avg_volts = avg_volts.reset_index()
avg_volts['Datetime'] = avg_volts['Date'].astype(str) + ' ' + avg_volts['Hour'].astype(str) + ':00:00'
avg_volts['Datetime'] = pd.to_datetime(avg_volts['Datetime'], format="%Y-%m-%d %H:%M:%S")

# -----------------------------

dfAvgHr = pd.DataFrame({'Timestamp': avg_volts['Datetime'], 
    'AvgTempBoiler': hr_avg['irfq::boiler:read_temp'],
    'AvgGasRead': hr_avg_g['irfq::h2_gas:read_flow'],
    'AvgVolts': avg_volts['irfq::ext:read_volts']})
dfAvgHr['Labels'] = 0
dfAvgHr

,Timestamp,AvgTempBoiler,AvgGasRead,AvgVolts,Labels
0,2023-09-04 08:00:00,163.277778,14.916667,17.042620,0
1,2023-09-04 09:00:00,162.947368,14.987500,17.073519,0
2,2023-09-04 10:00:00,161.975610,14.960606,16.662300,0
3,2023-09-04 11:00:00,161.991071,14.861404,17.192794,0
4,2023-09-04 12:00:00,162.000000,14.861290,14.415775,0
...,...,...,...,...,...
1172,2023-10-23 04:00:00,NaN,13.701695,17.447054,0
1173,2023-10-23 05:00:00,NaN,13.700000,17.406813,0
1174,2023-10-23 06:00:00,NaN,13.700943,17.415240,0
1175,2023-10-23 07:00:00,NaN,13.700000,17.405798,0


In [1863]:
for i in range(dfAvgHr.shape[0]-1):
    for el in failed['FaultDateTime']:
        if dfAvgHr['Timestamp'][i] <= el <= dfAvgHr['Timestamp'][i+1]:
            dfAvgHr.loc[i, 'Labels'] = 1
            break
labels = dfAvgHr['Labels'].tolist()
dfAvgHr.drop(['Labels'], axis=1, inplace=True)

In [1864]:
dfAvgHr.describe()

,AvgTempBoiler,AvgGasRead,AvgVolts
count,983.000000,1177.000000,1177.000000
mean,159.880594,17.091686,16.857478
std,19.143958,3.486041,2.318809
min,20.400000,0.867162,0.175140
25%,158.000000,14.975325,17.096646
50%,160.000000,15.497436,17.308513
75%,165.000000,19.812658,17.489852
max,548.250000,24.622340,18.051224


Start from behind, if you dont see a 1 in the labels in the feature window, continue, if you see move to the 1 and continue from there


In [1865]:
twindow = 12
tryout = dfAvgHr.copy()
tryout.dropna(inplace=True)
# for i in range(tryout.shape[0]-1,-1,-1):
# start = tryout.loc[0, 'Timestamp']
# tryout['Timestamp'] -= start
# # tryout
# #convert date time into hours
# tryout['Timestamp'] = tryout['Timestamp'].dt.total_seconds() / 3600
labels = labels[0:tryout.shape[0]]
for i in range(len(labels)):
    if labels[i] == 1:
        if i-twindow < 0:
            for j in range(0, i):
                labels[j] = 1
        else:
            for j in range(i-twindow+1, i):
                labels[j] = 1

In [1866]:
labels = pd.DataFrame(labels, columns=['Labels'])

In [1867]:
tryout.drop(['Timestamp'], axis=1, inplace=True)
tryout.drop(['AvgTempBoiler'], axis=1, inplace=True)

In [1868]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(tryout):
    X_train, X_test = tryout.iloc[train_index], tryout.iloc[test_index]
    y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

    rf = RandomForestClassifier(
        n_estimators=100,
        criterion='entropy',
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=True
    )

    rf.fit(X_train, y_train['Labels'])
    rf_pred = rf.predict(X_test)

    accuracy_test = accuracy_score(y_test, rf_pred)
    precision_test = precision_score(y_test, rf_pred, zero_division=1)
    recall_test = recall_score(y_test, rf_pred, zero_division=1)
    f1_test = f1_score(y_test, rf_pred, zero_division=1)
    conf_test = confusion_matrix(y_test, rf_pred)
    print(f"Accuracy: {accuracy_test}, Precision: {precision_test}, Recall: {recall_test}, F1: {f1_test}")
    print(conf_test)
    print(f"False labels in train set: {len(y_train) - sum(y_train['Labels'])}; True labels in train set: {sum(y_train['Labels'])}")
    print(f"False labels in test set: {len(y_test) - sum(y_test['Labels'])}; True labels in test set: {sum(y_test['Labels'])}")
    print(y_test['Labels'].to_list())
    print(rf_pred.tolist())
    print('--------------------------------------------------')

Accuracy: 0.852760736196319, Precision: 1.0, Recall: 0.0, F1: 0.0
[[139   0]
 [ 24   0]]
False labels in train set: 168; True labels in train set: 0
False labels in test set: 139; True labels in test set: 24
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0